Copyright © 2020, Weta Digital, Ltd.

SPDX-License-Identifier: Apache-2.0

# PhysLight Imaging

Here's a very simple idealized "renderer" that calculates the sRGB (linear) pixel values given a $2.5lx$ uniform environment light illuminating a 100% diffuse reflector.

We'll set default camera parameters according to the exposure equation and verify that our output pixel values are exactly 1.

In [51]:
!pip install -q colour-science
!pip install -q matplotlib

import colour
from colour import SpectralShape, CMFS
import numpy as np
import math


In [52]:

# Convert the given SpectralDistribution to XYZ using CIE 1931 2-degree,
# optionally normalizing such that Y=1.
# Note that we define this here rather than using colour's in-built functions
# for the sake of clarity
def spectral_to_XYZ(sd, normalize=False):
  cmf = CMFS['CIE 1931 2 Degree Standard Observer'].copy()
  x_bar = cmf.align(SpectralShape(360, 780, 1)).values[:,0]
  y_bar = cmf.align(SpectralShape(360, 780, 1)).values[:,1]
  z_bar = cmf.align(SpectralShape(360, 780, 1)).values[:,2]
  s = sd.copy().align(SpectralShape(360, 780, 1))
  nm = s.wavelengths

  x = np.trapz(x_bar * s.values, nm)
  y = np.trapz(y_bar * s.values, nm)
  z = np.trapz(z_bar * s.values, nm)

  if normalize:
    return [x, y, z] / y
  else:
    return [x, y, z]


We want to check our working against photometric quantities, to do this we'll want to normalize our light source such that its spectral distribution represents a luminance of $1 nit$. We do this by dividing by:

$$K_m \int_{360nm}^{780nm} S(\lambda) \bar{y}(\lambda) d\lambda$$


where $K_m = 683lm/W$. 


In [53]:
def to_photometric(sd):
  return spectral_to_XYZ(sd)[1] * 683

d65 = colour.ILLUMINANTS_SDS['D65'].copy()
spd_light = d65 / to_photometric(d65)

Our setup is $2.5lx$ incident on a 100% diffuse reflector. So exitant luminance from the surface, $L_v$ will be $\frac{2.5}{\pi} nit$

In [54]:
L_v = 2.5 / math.pi
# L is radiance scaled to $L_v nit$
L = spd_light * L_v

EV settings from Wikipedia $2.5lx$ $EV0$ example.


https://en.wikipedia.org/wiki/Exposure_value#Relationship_of_EV_to_lighting_conditions

When $EV=0$ (i.e. $2.5lx$ assuming $C=250$), then we should get a "correct" exposure
with these camera settings

In [55]:
t = 1.0
N = 1.0
S = 100.0
C = 250.0
K_m = 683.0


Convert radiance entering the camera system to exposure in $W m^{-2} nm^{-1} s$ (ish - we're actually representing some sort of output signal from the sensor here rather than exposure at the sensor, but it's easier to think of it this way)


In [56]:
imaging_ratio = (math.pi * t * S * K_m) / (C * N * N)
H = L * imaging_ratio

Convert to XYZ then to linear sRGB. We get back to exactly 1 in RGB by dividing by the normalized RGB whitepoint.

In [67]:
H_xyz = spectral_to_XYZ(H)

sRGB = colour.models.sRGB_COLOURSPACE

# We normalize when calculating the white point since we just want to affect
# colour, not brightness
white_xyz = spectral_to_XYZ(spd_light, normalize=True)
white_rgb = np.dot(sRGB.XYZ_to_RGB_matrix, white_xyz)

H_rgb = np.dot(sRGB.XYZ_to_RGB_matrix, H_xyz) / white_rgb

print('H_rgb', H_rgb)
assert np.array_equal(np.round(H_rgb, 15), [1.0, 1.0, 1.0])

H_rgb [ 1.  1.  1.]
